Este algoritmo cria uma coluna de correspondência para cada candidato e vaga. Em seguida, identifica os 3 maiores valores de "superfit" para cada empresa e os 4 maiores valores de "workstyle". A partir desses dados, ele calcula o valor do correspondência com base exclusivamente nas colunas de "workstyle" e "superfit" encontradas.

In [1]:
#Import das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.cluster import KMeans

In [3]:
#Precisamos calcular o match entre um candidato e empresa, então o nosso modelo compara cada candidato com cada empresa e com base na diferença entre os valores de superfit e lifestyle ele calcula a compatiblidade entre os dois.
# Inserção dos Dados de Usuários/Empresas
datauser = pd.read_csv('NORM_JUNCAO_USER.csv')
datajob = pd.read_csv('NORM_JUNCAO_JOB.csv')
datauser.drop('Unnamed: 0', axis=1, inplace=True)
datajob.drop('Unnamed: 0', axis=1, inplace=True)
datauser.describe()
#Adição de uma coluna chamada "Match", que mostrará o quanto o candidato combina com a empresa
datauser['match'] = 1
datajob['match'] = 1
combinations = pd.merge(datauser, datajob, on='match')

#Features escolhidas: As 3 colunas do superfit que a empresa mais julga importante (os 3 valores mais altos) e as as 4 colunas do lifestyle que a empresa mais julga importante (os 4 valores mais altos)
#Justificativa: Como não foi fornecido um "peso" para cada variável (apenas o peso de superfit e lifestyle), foi decidido que as variáveis que usaremos será as que tiverem o maior valor numérico, sendo inferido que aquela habilidade seja considerada mais importante pela empresa
df = combinations

#restringe o dataframe para pegar só as habilidades do superfit
job_superfit = df[['score_res','score_eng', 'score_int','score_cur','score_sin','score_dis']]

#função que pega os 3 valores mais altos do superfit de todas as empresas e guarda o nome das colunas em um array
def top_3_maiores_indices_superfit(dataframe):
    # converte o dataframe para numpy
    data_array = dataframe.to_numpy()
    # acha os índices dos 3 valores mais altos
    top_3_indices_superfit = np.argsort(-data_array, axis=1)[:, :3]
    # converte o dataframe para um array do numpy
    columns_as_array = np.array(dataframe.columns)
    # usa os índices para achar o nome das colunas e guardar elas numa variável
    top_3_column_names = columns_as_array[top_3_indices_superfit]
    # retorna a variável com o array com as 3 habilidades mais importantes para cada empresa
    return top_3_column_names

#chama a função supracitada e usa o job_superfit como o dataframe
top_3_indices_superfit = top_3_maiores_indices_superfit(job_superfit)
#printa a variável
print(top_3_indices_superfit)

[['score_res' 'score_eng' 'score_dis']
 ['score_res' 'score_int' 'score_dis']
 ['score_res' 'score_sin' 'score_int']
 ...
 ['score_res' 'score_eng' 'score_cur']
 ['score_dis' 'score_res' 'score_eng']
 ['score_res' 'score_dis' 'score_sin']]


In [4]:
#restringe o dataframe para pegar só as habilidades do workstyle
job_workstyle = df[['score_classic', 'score_order', 'score_change', 'score_tireless', 'score_explorer', 'score_specialist', 'score_generalist']]

#função que pega os 4 valores mais altos do workstyle de todas as empresas e guarda o nome das colunas em um array
def top_4_maiores_indices_workstyle(dataframe):
    # converte o dataframe para numpy
    data_array = dataframe.to_numpy()
    # acha os índices dos 4 valores mais altos
    top_4_indices_workstyle = np.argsort(-data_array, axis=1)[:, :4]
    # converte o dataframe para um array do numpy
    columns_as_array = np.array(dataframe.columns)
    # usa os índices para achar o nome das colunas e guardar elas numa variável
    top_4_column_names_workstyle = columns_as_array[top_4_indices_workstyle]
    # retorna a variável com o array com as 4 habilidades mais importantes para cada empresa
    return top_4_column_names_workstyle

#chama a função supracitada e usa o job_workstyle como o dataframe
top_4_indices_workstyle = top_4_maiores_indices_workstyle(job_workstyle)
#printa a variável
print(top_4_indices_workstyle)

[['score_specialist' 'score_order' 'score_generalist' 'score_tireless']
 ['score_generalist' 'score_explorer' 'score_tireless' 'score_change']
 ['score_tireless' 'score_generalist' 'score_explorer' 'score_specialist']
 ...
 ['score_specialist' 'score_explorer' 'score_change' 'score_generalist']
 ['score_generalist' 'score_explorer' 'score_tireless' 'score_classic']
 ['score_specialist' 'score_explorer' 'score_generalist' 'score_change']]


In [5]:
#função que usa o nome das colunas do dataframe das empresas para achar a coluna relacionada no dataframe do usuário
def get_related_columns(coluna1, coluna2, coluna3, coluna4, coluna5, coluna6, coluna7):
    #define as correlações
    correlacoes = {
        'score_res': 'superfit_res',
        'score_eng': 'superfit_eng',
        'score_int': 'superfit_int',
        'score_cur': 'superfit_cur',
        'score_sin': 'superfit_sin',
        'score_dis': 'superfit_dis',
        'score_classic': 'lifestyle_classic',
        'score_order': 'lifestyle_order',
        'score_change': 'lifestyle_change',
        'score_tireless': 'lifestyle_tireless',
        'score_explorer': 'lifestyle_explorer',
        'score_specialist': 'lifestyle_specialist',
        'score_generalist': 'lifestyle_generalist'
    }
    #retorna a correlação das variáveis coluna de 1 a 7 e salva essas correlações nas variáveis coluna_relacionada de 1 a 7
    return correlacoes[coluna1], correlacoes[coluna2], correlacoes[coluna3], correlacoes[coluna4], correlacoes[coluna5], correlacoes[coluna6], correlacoes[coluna7]

#função para calcular o match do usuário com emrpesas
def calculate_match_score(df, specific_id, top_3_indices_superfit, top_4_indices_workstyle, number):
    # Filtrar o DataFrame pelo ID específico
    df_filtered = df[df['id'] == specific_id]
    # Verificar se o ID existe no DataFrame
    if df_filtered.empty:
        return None
    # Pegar os índices das colunas superfit e workstyle com base nos top 3 e top 4 índices
    coluna1, coluna2, coluna3 = top_3_indices_superfit[specific_id]
    coluna4, coluna5, coluna6, coluna7 = top_4_indices_workstyle[specific_id]
    #chama a função get_related_columns para achar as colunas relacionadas (nome da coluna, mas referente as skills do usuário)
    coluna_relacionada1, coluna_relacionada2, coluna_relacionada3, coluna_relacionada4, coluna_relacionada5, coluna_relacionada6, coluna_relacionada7 = get_related_columns(coluna1, coluna2, coluna3, coluna4, coluna5, coluna6, coluna7)
    # Calcular as médias para superfit e lifestyle
    mediasSuperfit = (abs(df[coluna1] - df[coluna_relacionada1]) + abs(df[coluna2] - df[coluna_relacionada2]) + abs(df[coluna3] - df[coluna_relacionada3])) / 3
    mediasLifestyle = (abs(df[coluna4] - df[coluna_relacionada4]) + abs(df[coluna5] - df[coluna_relacionada5]) + abs(df[coluna6] - df[coluna_relacionada6]) + abs(df[coluna7] - df[coluna_relacionada7])) / 4
    # Calcular o match score
    df_filtered['match'] = 1 - (mediasSuperfit + mediasLifestyle) / 2
    # Filtrar e classificar os resultados
    df_filtered = df_filtered.nlargest(number, 'match')[['id', 'job_opportunity_id', 'match']]
    #retorna o resultado dos matchs
    return df_filtered

#chama a função para achar o match. Usa 2 números que variam: o segundo parâmetro que seria o ID da pessoa e o quinto, que seria quantos matchs irá retornar ao usuário
calculate_match_score(df, 1055906, top_3_indices_superfit, top_4_indices_workstyle, 10)


C:\Users\mario\AppData\Local\Temp\ipykernel_26888\3939590223.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['match'] = 1 - (mediasSuperfit + mediasLifestyle) / 2


,id,job_opportunity_id,match
3877311,1055906,2909,0.928737
3877194,1055906,2781,0.887088
3877261,1055906,2841,0.879509
3877301,1055906,2890,0.869688
3877212,1055906,2888,0.864198
3877265,1055906,2772,0.851665
3877323,1055906,2894,0.849130
3877319,1055906,2957,0.846695
3877303,1055906,2884,0.844886
3877199,1055906,2775,0.843954
